In [1]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
import fitz  # PyMuPDF
import openai
from dotenv import load_dotenv
from langchain.embeddings import HuggingFaceEmbeddings
import ipywidgets as widgets
from IPython.display import display, Markdown
from PyPDF2 import PdfReader
import os
from langchain.chat_models import ChatOpenAI

In [2]:
import ipywidgets as widgets
from IPython.display import display, Markdown
from dotenv import load_dotenv
from openai import OpenAI
import os

load_dotenv()


True

1. Extract Text from the PDFs:

In [3]:
# Function to read PDF and extract text
def extract_pdf_text(pdf_path):
    reader = PdfReader(pdf_path)
    pdf_text = ""
    for page in reader.pages:
        pdf_text += page.extract_text()
    return pdf_text


# Function to create vector store from smaller chunks of PDF content
def create_vector_store_from_pdf(pdf_text):
    # Split the extracted text into smaller, manageable chunks
    chunk_size = 1000  # Set chunk size to be smaller than the token limit
    chunks = [pdf_text[i:i+chunk_size] for i in range(0, len(pdf_text), chunk_size)]
    
    # Create embeddings for the text chunks
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")  # Adjust model as needed
    vector_store = FAISS.from_texts(chunks, embeddings)
    
    return vector_store

    

In [ ]:
# def extract_accident_law_from_pdf(pdf_path, first_relevant_page=0):
#     doc = fitz.open(pdf_path)
#     law_text = []  # Store the extracted text
    
#     for page_num, page in enumerate(doc):
#         if page_num < first_relevant_page:
#             continue
        
#         page_text = page.get_text() 
#         law_text.append(page_text)  
    
#     return law_text


# # Example usage
# # pdf_path = "path_to_your_accident_law_pdf.pdf"
# doc2_path = "Doc2.pdf"
# accident_law_text = extract_accident_law_from_pdf(doc2_path, first_relevant_page=2)
# print(accident_law_text[:1000])  # Print the first 1000 characters for inspection


[" \n3\nAlomar,                                                                                                         \n \nاﻟﺮﻗــــﻢ:      /............................................ \nاﻟﺘﺎر*ﺦ:........................................................ \nاﻟﺘﻮا-ﻊ:........................................................ \nاﻹدارة اﻟﻌﺎﻣﺔ ﻟﻠﺸﺌﻮن اﳌﺎﻟﻴﺔ وا45ﺪﻣﺎت اﳌﺴﺎﻧﺪة \nاﻹدارة اﻟﻌﺎﻣﺔ ﻟﻠ:;ﺟﻤﺔ \n \n12. Stopping or Waiting: stopping vehicles somewhere for a limited or \nunlimited period in other than stopping conditions. \n13. Parking: a place specified for vehicles to park. \n14. Road Signs: lines or signs on the pathway or on its both sides for \norganizing the use of the pathway. \n15. Reflectors: any reflector material placed on the pathway as one of its \nsigns or placed in either sides or one side of  a vehicle which can seen \nfrom a distance.  \n16. Turn: a curve changes the pathway the direction.  \n17. Traffic Signs: every signs and signals founded on the pathway and \nothers. \n18

2. Split into chunks

In [4]:
# Fixed-size chunking strategy using RecursiveCharacterTextSplitter
def chunk_text(text, chunk_size=500, chunk_overlap=200):
    # Create the text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )
    # Split the text into chunks
    chunks = text_splitter.split_text(text)
    return chunks


doc2_path = "Doc2.pdf"
accident_law_text = extract_accident_law_from_pdf(doc2_path, first_relevant_page=2)

# Combine all extracted text into one string
combined_text = "\n".join(accident_law_text)

# Now chunk the combined text
chunks = chunk_text(combined_text)

print(chunks[1])  # Print the first chunk


NameError: name 'extract_accident_law_from_pdf' is not defined

3. Create embeddings and vector store

In [5]:


load_dotenv()


# Function to create embeddings and store them in a vector store
def create_embeddings_and_vector_store(chunks):
    # Initialize OpenAI Embeddings
    #openai.api_key = 'your-openai-api-key'
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    
    # Create the vector store using FAISS
    vector_store = FAISS.from_texts(chunks, embeddings)
    return vector_store

# Create vector store with the chunks
vector_store = create_embeddings_and_vector_store(chunks)


NameError: name 'chunks' is not defined

4. Create RAG chain

In [6]:
def create_rag_chain(vector_store):
    
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

    # Create a RetrievalQA chain with "stuff" type
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vector_store.as_retriever(search_kwargs={"k": 4})  # Retrieve top 4 relevant chunks
    )
    return qa_chain

# Create the RAG chain using your vector store
rag_chain = create_rag_chain(vector_store)

NameError: name 'vector_store' is not defined

5. Evaluation & Quality Assurance

In [7]:
def basic_health_check(vector_store, test_queries):
    results = {}
    for query in test_queries:
        try:
            # Retrieve documents
            docs = vector_store.similarity_search(query, k=4)
            # Store the number of retrieved documents
            results[query] = {"retrieved_docs_count": len(docs)}
        except Exception as e:
            results[query] = {"error": str(e)}
    return results

test_queries = [
    "I turned on the car, and was getting ready to leave the parking space (it was a 90-degree parking spot). I put the car in reverse (R) and started backing up, but I didn’t reverse far enough. As I straightened the car to move forward, I accidentally bumped into the car parked in the spot next to mine., who is responsible for the accident?"
]
health_check_results = basic_health_check(vector_store, test_queries)
print(health_check_results)


NameError: name 'vector_store' is not defined

In [20]:
import os
from PyPDF2 import PdfReader
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
import ipywidgets as widgets
from IPython.display import display, Markdown

# Function to read PDF and extract text
def extract_pdf_text(pdf_path):
    reader = PdfReader(pdf_path)
    pdf_text = ""
    for page in reader.pages:
        pdf_text += page.extract_text()
    return pdf_text

# Function to create vector store from smaller chunks of PDF content
def create_vector_store_from_pdf(pdf_text):
    chunk_size = 1000  # Set chunk size to be smaller than the token limit
    chunks = [pdf_text[i:i+chunk_size] for i in range(0, len(pdf_text), chunk_size)]
    
    # Create embeddings for the text chunks
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")  # Adjust model as needed
    vector_store = FAISS.from_texts(chunks, embeddings)
    
    return vector_store

# Function to create RAG chain
def create_rag_chain(vector_store):
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)  # Adjust model as needed
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vector_store.as_retriever(search_kwargs={"k": 1})  # Retrieve top 1 relevant chunk
    )
    return qa_chain


# Function to collect Driver 1's information and accident description
def collect_driver_1_info(vector_store):
    case_id_widget = widgets.Text(description="Case ID:")
    date_widget = widgets.DatePicker(description="Accident Date:")
    driver1_name_widget = widgets.Text(description="Driver 1 Name:")
    driver1_car_plate_widget = widgets.Text(description="Driver 1 Car Plate:")
    driver1_accident_desc_widget = widgets.Textarea(description="Driver 1 Accident Description:", layout=widgets.Layout(width="100%", height="200px"))
    
    display(Markdown("### Driver 1 Information"))
    display(case_id_widget, date_widget, driver1_name_widget, driver1_car_plate_widget, driver1_accident_desc_widget)

    def on_submit_driver_1(sender):
        driver_1_info = {
            "case_id": case_id_widget.value,
            "accident_date": date_widget.value,
            "driver1_name": driver1_name_widget.value,
            "driver1_car_plate": driver1_car_plate_widget.value,
            "driver1_accident_desc": driver1_accident_desc_widget.value
        }
        
        display(Markdown("**Driver 1 Information Collected**"))
        display(driver_1_info)
        
        collect_driver_2_info(driver_1_info, vector_store)
    
    submit_button_driver_1 = widgets.Button(description="Submit Driver 1 Info")
    submit_button_driver_1.on_click(on_submit_driver_1)
    display(submit_button_driver_1)

# Function to collect Driver 2's information and accident description
def collect_driver_2_info(driver_1_info, vector_store):
    driver2_name_widget = widgets.Text(description="Driver 2 Name:")
    driver2_car_plate_widget = widgets.Text(description="Driver 2 Car Plate:")
    driver2_accident_desc_widget = widgets.Textarea(description="Driver 2 Accident Description:", layout=widgets.Layout(width="100%", height="200px"))
    
    display(Markdown("### Driver 2 Information"))
    display(driver2_name_widget, driver2_car_plate_widget, driver2_accident_desc_widget)

    def on_submit_driver_2(sender):
        driver_2_info = {
            "driver2_name": driver2_name_widget.value,
            "driver2_car_plate": driver2_car_plate_widget.value,
            "driver2_accident_desc": driver2_accident_desc_widget.value
        }
        
        display(Markdown("**Driver 2 Information Collected**"))
        display(driver_2_info)
        
        # Now evaluate the case based on both drivers' descriptions using RAG chain
        evaluate_case(driver_1_info, driver_2_info, vector_store)
    
    submit_button_driver_2 = widgets.Button(description="Submit Driver 2 Info")
    submit_button_driver_2.on_click(on_submit_driver_2)
    display(submit_button_driver_2)

# Function to evaluate the case based on both drivers' accident descriptions and the law
def evaluate_case(driver_1_info, driver_2_info, vector_store):
    # Shorten the case query if needed to fit within token limits
    case_query = f"Case {driver_1_info['case_id']}: Driver 1 ({driver_1_info['driver1_name']} - {driver_1_info['driver1_car_plate']}) and Driver 2 ({driver_2_info['driver2_name']} - {driver_2_info['driver2_car_plate']}) were involved in an accident on {driver_1_info['accident_date']}. "
    case_query += f"Driver 1: {driver_1_info['driver1_accident_desc'][:500]} ... Driver 2: {driver_2_info['driver2_accident_desc'][:500]} ... Who is responsible for the accident?"
    
    # Create the RAG chain
    rag_chain = create_rag_chain(vector_store)
    
    # Get the response from the RAG chain
    try:
        response = rag_chain.run(case_query)
        display(Markdown(f"**RAG Decision:** {response}"))
        
        # After RAG decision, ask for agreement from both drivers
        collect_agreement(driver_1_info, driver_2_info, response)
    except Exception as e:
        display(Markdown(f"Error in RAG decision: {str(e)}"))

# Function to collect agreement from both drivers
def collect_agreement(driver_1_info, driver_2_info, decision):
    # Agreement for Driver 1
    driver1_agree_widget = widgets.RadioButtons(
        options=['Yes', 'No'],
        description=f"Driver 1 agree with decision: '{decision}'?",
        disabled=False
    )
    
    # Agreement for Driver 2
    driver2_agree_widget = widgets.RadioButtons(
        options=['Yes', 'No'],
        description=f"Driver 2 agree with decision: '{decision}'?",
        disabled=False
    )

    # Function to handle the driver's agreement responses
    def on_agreement_change(change):
        driver1_agreement = driver1_agree_widget.value
        driver2_agreement = driver2_agree_widget.value
        
        # Check if both drivers agree
        if driver1_agreement == 'Yes' and driver2_agreement == 'Yes':
            display(Markdown("Both drivers agree with the decision. The case is closed."))
        else:
            display(Markdown("There is a contradiction. Please call the police."))

    # Observe both widgets for changes
    driver1_agree_widget.observe(on_agreement_change, names='value')
    driver2_agree_widget.observe(on_agreement_change, names='value')
    
    display(driver1_agree_widget)
    display(driver2_agree_widget)

# Read and process the PDF to create a vector store
pdf_path = "Doc2.pdf"  # Path to your PDF file
pdf_text = extract_pdf_text(pdf_path)

# Create the vector store from the PDF content
vector_store = create_vector_store_from_pdf(pdf_text)

# Start the collection of Driver 1 information
collect_driver_1_info(vector_store)


### Driver 1 Information

Text(value='', description='Case ID:')

DatePicker(value=None, description='Accident Date:', step=1)

Text(value='', description='Driver 1 Name:')

Text(value='', description='Driver 1 Car Plate:')

Textarea(value='', description='Driver 1 Accident Description:', layout=Layout(height='200px', width='100%'))

Button(description='Submit Driver 1 Info', style=ButtonStyle())

In [9]:
def evaluate_rag_chain(rag_chain, test_queries):
    evaluation_results = {}
    for query in test_queries:
        try:
            # Get response from RAG chain
            response = rag_chain.run(query)
            evaluation_results[query] = {"response": response}
        except Exception as e:
            evaluation_results[query] = {"error": str(e)}
    return evaluation_results
# Example usage:
qa_chain = create_rag_chain(vector_store)  # Your RAG chain
evaluation_results = evaluate_rag_chain(qa_chain, test_queries)
for query, result in evaluation_results.items():
    print(f"\n Query: {query}")
    print(f"Response: {result.get('response', 'Error occurred')}")

/var/folders/0d/yb3d6xxj1qs2wnw40gz9y3tc0000gn/T/ipykernel_86044/2348865514.py:22: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)  # Adjust model as needed
/var/folders/0d/yb3d6xxj1qs2wnw40gz9y3tc0000gn/T/ipykernel_86044/4017630078.py:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = rag_chain.run(query)



 Query: I turned on the car, and was getting ready to leave the parking space (it was a 90-degree parking spot). I put the car in reverse (R) and started backing up, but I didn’t reverse far enough. As I straightened the car to move forward, I accidentally bumped into the car parked in the spot next to mine., who is responsible for the accident?
Response: Based on the provided information, it seems that the responsibility for the accident would likely fall on you as the driver who was backing out of the parking space. It is important for drivers to ensure that they can make movements without endangering themselves or other road users, as mentioned in the driving regulations. Additionally, giving the necessary signals and being aware of the surroundings is crucial to avoid accidents like the one described.


In [10]:
def query_with_traceability(chain, query):
    # Run the chain with the query
    response = chain.run(query)
    # Retrieve the related documents
    retrieved_docs = chain.retriever.get_relevant_documents(query)
    # Print the retrieved documents and their metadata
    print("\n--- Retrieved Chunks ---")
    for doc in retrieved_docs:
        print(f"Source Metadata: {doc.metadata}")
        print(f"Content Preview: {doc.page_content[:500]}...\n")
    # Print the response
    print("\n--- Model Response ---")
    print(response)
    return response


response = query_with_traceability(qa_chain, query)

/var/folders/0d/yb3d6xxj1qs2wnw40gz9y3tc0000gn/T/ipykernel_86044/1763416274.py:5: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = chain.retriever.get_relevant_documents(query)



--- Retrieved Chunks ---
Source Metadata: {}
Content Preview:  
 
50/2/23- The driver of the vehicle must take the right side of the road in the following cases: 50/2/23/1- If he/she is turning right to another road. 50/2/23/2- If there is another vehicle, he will bypass. 50/2/23/3- If his/her vehicle speed is less than the authorized maximum speed. 50/2/23/4 - When the vision is not clear on the road. 50/2/23/5- If he/she is facing another vehicle coming from the opposite direction. 50/2/24- Every driver, before making any movement related to turning his/...


--- Model Response ---
Based on the provided information, it seems that the driver who was backing out of the parking space and accidentally bumped into the car parked in the spot next to theirs would likely be responsible for the accident. This is because the driver is expected to ensure that any movement related to turning the car is done without exposing themselves or other road users to danger, as per the driving regulation